# Create Datasets with Stratified Sampling and Augmentation

This notebook creates 4 dataset sizes (1.5h, 2h, 4h, 6h) with:
- Stratified sampling (40% Mekdes, 40% Ameha, 20% gTTS; domain proportions maintained)
- Audio augmentation (2x factor)
- Train/Val/Test split (80/10/10)
- CSV files ready for training notebooks

## 1. Installation and Imports


In [11]:
%pip install -q librosa soundfile numpy pandas scipy



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
import shutil
from tqdm import tqdm
import random
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)


## 2. Configuration


In [13]:
# Paths
METADATA_FILE = "Dataset/audio/metadata.csv"
AUDIO_DIR = Path("Dataset/audio")

# Target dataset sizes (hours AFTER augmentation)
TARGET_SIZES_HOURS = [1.5, 2.0, 4.0, 6.0]

# Augmentation factor (2x = one augmented version per original)
AUGMENTATION_FACTOR = 2

# Voice distribution (must sum to 1.0)
VOICE_DISTRIBUTION = {
    "mekdes": 0.40,
    "ameha": 0.40,
    "gtts": 0.20
}

# Split ratios
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

# Sampling rate
SR = 16000

print("Configuration:")
print(f"  Target sizes (after augmentation): {TARGET_SIZES_HOURS} hours")
print(f"  Augmentation factor: {AUGMENTATION_FACTOR}x")
print(f"  Voice distribution: {VOICE_DISTRIBUTION}")
print(f"  Split ratios: Train={TRAIN_RATIO}, Val={VAL_RATIO}, Test={TEST_RATIO}")


Configuration:
  Target sizes (after augmentation): [1.5, 2.0, 4.0, 6.0] hours
  Augmentation factor: 2x
  Voice distribution: {'mekdes': 0.4, 'ameha': 0.4, 'gtts': 0.2}
  Split ratios: Train=0.8, Val=0.1, Test=0.1


## 3. Load Metadata


In [14]:
df = pd.read_csv(METADATA_FILE)

print(f"Total samples in metadata: {len(df)}")
print(f"Total duration: {df['duration'].sum() / 3600:.2f} hours")
print(f"\nVoice distribution:")
print(df['voice'].value_counts())
print(f"\nDomain distribution:")
print(df['domain_name'].value_counts())


Total samples in metadata: 7590
Total duration: 25.04 hours

Voice distribution:
mekdes    3036
ameha     3036
gtts      1518
Name: voice, dtype: int64

Domain distribution:
Criminal Law and Procedures                 1188
Contracts and Commercial Law                1041
Constitutional and Administrative Law       1033
Tax and Financial Law                        877
Property and Civil Law                       812
Regulatory and Administrative Procedures     774
Court Procedures and Judicial Processes      702
Family Law and Labor Law                     667
International Law and Treaties               496
Name: domain_name, dtype: int64


## 4. Augmentation Functions


In [15]:
def add_gaussian_noise(audio, noise_factor=0.005):
    """Add Gaussian white noise"""
    noise = np.random.normal(0, noise_factor, audio.shape)
    augmented = audio + noise
    return np.clip(augmented, -1.0, 1.0)

def time_stretch(audio, rate=0.95):
    """Time stretch (speed variation)"""
    return librosa.effects.time_stretch(audio, rate=rate)

def pitch_shift(audio, sr, n_steps=2):
    """Pitch shift (disabled due to numpy compatibility)"""
    return audio  # Disabled
def volume_change(audio, gain_db=3.0):
    """Change volume (amplitude scaling)"""
    gain_linear = 10 ** (gain_db / 20.0)
    augmented = audio * gain_linear
    return np.clip(augmented, -1.0, 1.0)

def apply_reverb(audio, room_size=0.3, damping=0.5, wet_level=0.1):
    """Simple reverb simulation using convolution with impulse response"""
    # Create a simple impulse response
    ir_length = int(0.1 * len(audio))  # 10% of audio length
    impulse_response = np.exp(-np.arange(ir_length) * damping / room_size)
    impulse_response = impulse_response / np.sum(impulse_response)
    
    # Convolve audio with impulse response
    reverb_signal = signal.convolve(audio, impulse_response, mode='same')
    
    # Mix dry and wet signals
    augmented = audio + reverb_signal * wet_level
    return np.clip(augmented, -1.0, 1.0)

def augment_audio(audio, sr, augmentation_type='random'):
    """Apply one augmentation to audio"""
    if augmentation_type == 'random':
        # Removed pitch_shift due to numpy compatibility issues
        augmentation_type = random.choice(['noise', 'time_stretch', 'volume', 'reverb'])
    
    if augmentation_type == 'noise':
        return add_gaussian_noise(audio, noise_factor=random.uniform(0.003, 0.01))
    elif augmentation_type == 'time_stretch':
        rate = random.uniform(0.9, 1.1)
        return time_stretch(audio, rate=rate)
    elif augmentation_type == 'pitch_shift':
        # Skip pitch_shift due to numpy compatibility issues
        return audio
    elif augmentation_type == 'volume':
        gain_db = random.uniform(-3.0, 3.0)
        return volume_change(audio, gain_db=gain_db)
    elif augmentation_type == 'reverb':
        return apply_reverb(audio, 
                           room_size=random.uniform(0.2, 0.5),
                           wet_level=random.uniform(0.05, 0.15))
    else:
        return audio

print("Augmentation functions defined")


Augmentation functions defined


## 5. Stratified Sampling Function

In [16]:
def stratified_sample(df, target_duration_hours, voice_distribution, random_seed=42):
    """
    Stratified sampling maintaining voice and domain proportions
    
    Args:
        df: DataFrame with columns: file_name, transcription, domain_name, voice, duration
        target_duration_hours: Target total duration in hours (BEFORE augmentation)
        voice_distribution: Dict with voice proportions (must sum to 1.0)
        random_seed: Random seed for reproducibility
    
    Returns:
        Sampled DataFrame
    """
    target_duration_seconds = target_duration_hours * 3600
    
    # Set random seed
    random.seed(random_seed)
    np.random.seed(random_seed)
    
    sampled_records = []
    current_duration = 0.0
    
    # Calculate target duration per voice
    voice_targets = {voice: target_duration_seconds * prop for voice, prop in voice_distribution.items()}
    voice_current = {voice: 0.0 for voice in voice_distribution.keys()}
    
    # Calculate domain proportions from full dataset
    domain_counts = df['domain_name'].value_counts()
    domain_proportions = domain_counts / domain_counts.sum()
    
    # Group by voice and domain
    grouped = df.groupby(['voice', 'domain_name'])
    
    # Create pools for each voice-domain combination
    pools = {}
    for (voice, domain), group_df in grouped:
        pools[(voice, domain)] = group_df.copy().reset_index(drop=True)
    
    # Shuffle each pool
    for key in pools:
        pools[key] = pools[key].sample(frac=1, random_state=random_seed).reset_index(drop=True)
    
    # Track indices for each pool
    pool_indices = {key: 0 for key in pools}
    
    # Sample iteratively, maintaining proportions
    max_iterations = len(df) * 2  # Safety limit
    iteration = 0
    
    while current_duration < target_duration_seconds and iteration < max_iterations:
        iteration += 1
        
        # Determine which voice needs more samples (greedy approach)
        voice_deficits = {v: voice_targets[v] - voice_current[v] for v in voice_distribution.keys()}
        
        # Select voice with largest deficit
        selected_voice = max(voice_deficits, key=voice_deficits.get)
        
        if voice_deficits[selected_voice] <= 0:
            # All voice targets met, sample proportionally
            selected_voice = random.choices(
                list(voice_distribution.keys()),
                weights=list(voice_distribution.values()),
                k=1
            )[0]
        
        # Select domain proportionally for this voice
        available_domains = [d for (v, d) in pools.keys() if v == selected_voice]
        if not available_domains:
            break
        
        # Use domain proportions
        domain_weights = [domain_proportions.get(d, 0) for d in available_domains]
        if sum(domain_weights) == 0:
            selected_domain = random.choice(available_domains)
        else:
            selected_domain = random.choices(available_domains, weights=domain_weights, k=1)[0]
        
        pool_key = (selected_voice, selected_domain)
        
        # Get next sample from this pool
        if pool_indices[pool_key] >= len(pools[pool_key]):
            # Pool exhausted, try another domain for this voice
            continue
        
        sample = pools[pool_key].iloc[pool_indices[pool_key]]
        pool_indices[pool_key] += 1
        
        # Check if we should add this sample
        sample_duration = sample['duration']
        
        if current_duration + sample_duration > target_duration_seconds:
            # Would exceed target, check if we're close enough
            if current_duration / target_duration_seconds >= 0.95:
                break
            continue
        
        # Add sample
        sampled_records.append(sample)
        current_duration += sample_duration
        voice_current[selected_voice] += sample_duration
    
    result_df = pd.DataFrame(sampled_records).reset_index(drop=True)
    
    return result_df

print("Stratified sampling function defined")


Stratified sampling function defined


## 6. Dataset Creation Function

In [17]:
def create_dataset(target_size_hours, output_dir_name, augmentation_factor=2):
    """
    Create a dataset with stratified sampling and augmentation
    
    Args:
        target_size_hours: Target size AFTER augmentation
        output_dir_name: Name of output directory (e.g., 'Dataset_1.5h')
        augmentation_factor: How many augmented versions per original (default 2)
    """
    print(f"\n{'='*70}")
    print(f"Creating dataset: {output_dir_name} ({target_size_hours}h AFTER augmentation)")
    print(f"{'='*70}")
    
    # Calculate target BEFORE augmentation
    target_before_aug = target_size_hours / augmentation_factor
    print(f"Target before augmentation: {target_before_aug:.2f} hours")
    
    # Stratified sampling
    print("\nPerforming stratified sampling...")
    sampled_df = stratified_sample(df, target_before_aug, VOICE_DISTRIBUTION)
    
    actual_duration = sampled_df['duration'].sum() / 3600
    print(f"Sampled {len(sampled_df)} files, {actual_duration:.2f} hours")
    print(f"Voice distribution in sampled data:")
    print(sampled_df['voice'].value_counts())
    
    # Create output directory
    output_dir = Path(output_dir_name)
    output_audio_dir = output_dir / "audio"
    output_audio_dir.mkdir(parents=True, exist_ok=True)
    
    # Copy original files and create augmented versions
    print(f"\nCopying and augmenting audio files...")
    augmented_records = []
    file_counter = 0
    
    for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc="Processing"):
        original_file = AUDIO_DIR / row['file_name']
        
        if not original_file.exists():
            print(f"Warning: File not found: {original_file}")
            continue
        
        # Copy original file
        new_filename = f"{file_counter:06d}_{row['voice']}.wav"
        new_filepath = output_audio_dir / new_filename
        shutil.copy2(original_file, new_filepath)
        
        augmented_records.append({
            'file_name': new_filename,
            'transcription': row['transcription'],
            'domain_num': row['domain_num'],
            'domain_name': row['domain_name'],
            'voice': row['voice'],
            'duration': row['duration'],
            'is_augmented': False
        })
        file_counter += 1
        
        # Create augmented versions
        try:
            audio, sr = librosa.load(original_file, sr=SR)
            
            for aug_idx in range(augmentation_factor - 1):
                # Apply augmentation
                augmented_audio = augment_audio(audio, sr, augmentation_type='random')
                
                # Calculate duration
                aug_duration = len(augmented_audio) / sr
                
                # Save augmented audio
                aug_filename = f"{file_counter:06d}_{row['voice']}_aug{aug_idx+1}.wav"
                aug_filepath = output_audio_dir / aug_filename
                
                sf.write(str(aug_filepath), augmented_audio, SR)
                
                augmented_records.append({
                    'file_name': aug_filename,
                    'transcription': row['transcription'],
                    'domain_num': row['domain_num'],
                    'domain_name': row['domain_name'],
                    'voice': row['voice'],
                    'duration': aug_duration,
                    'is_augmented': True
                })
                file_counter += 1
                
        except Exception as e:
            print(f"Error processing {original_file}: {e}")
            continue
    
    augmented_df = pd.DataFrame(augmented_records)
    final_duration = augmented_df['duration'].sum() / 3600
    
    print(f"\nAugmented dataset: {len(augmented_df)} files, {final_duration:.2f} hours")
    print(f"Original: {len(augmented_df[~augmented_df['is_augmented']])} files")
    print(f"Augmented: {len(augmented_df[augmented_df['is_augmented']])} files")
    
    # Split into train/val/test
    print(f"\nSplitting into train/val/test (80/10/10)...")
    
    # Shuffle dataframe
    augmented_df = augmented_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    n_total = len(augmented_df)
    n_train = int(n_total * TRAIN_RATIO)
    n_val = int(n_total * VAL_RATIO)
    
    train_df = augmented_df.iloc[:n_train].copy()
    val_df = augmented_df.iloc[n_train:n_train+n_val].copy()
    test_df = augmented_df.iloc[n_train+n_val:].copy()
    
    # Create CSV files with only file_name and transcription
    train_csv = train_df[['file_name', 'transcription']].copy()
    val_csv = val_df[['file_name', 'transcription']].copy()
    test_csv = test_df[['file_name', 'transcription']].copy()
    
    # Save CSV files
    train_csv_path = output_dir / "train.csv"
    val_csv_path = output_dir / "val.csv"
    test_csv_path = output_dir / "test.csv"
    
    train_csv.to_csv(train_csv_path, index=False, encoding='utf-8')
    val_csv.to_csv(val_csv_path, index=False, encoding='utf-8')
    test_csv.to_csv(test_csv_path, index=False, encoding='utf-8')
    
    print(f"\nDataset created in: {output_dir}")
    print(f"  Train: {len(train_csv)} samples ({train_df['duration'].sum()/3600:.2f}h)")
    print(f"  Val: {len(val_csv)} samples ({val_df['duration'].sum()/3600:.2f}h)")
    print(f"  Test: {len(test_csv)} samples ({test_df['duration'].sum()/3600:.2f}h)")
    print(f"  Total: {len(augmented_df)} samples ({final_duration:.2f}h)")
    
    return augmented_df, train_df, val_df, test_df

print("Dataset creation function defined")


Dataset creation function defined


## 7. Create All Datasets

In [18]:
results = {}

for target_size in TARGET_SIZES_HOURS:
    output_dir_name = f"Dataset_{target_size}h"
    
    try:
        augmented_df, train_df, val_df, test_df = create_dataset(
            target_size, 
            output_dir_name, 
            augmentation_factor=AUGMENTATION_FACTOR
        )
        results[target_size] = {
            'augmented_df': augmented_df,
            'train_df': train_df,
            'val_df': val_df,
            'test_df': test_df
        }
    except Exception as e:
        print(f"\nError creating dataset {target_size}h: {e}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*70}")
print("All datasets created successfully!")
print(f"{'='*70}")



Creating dataset: Dataset_1.5h (1.5h AFTER augmentation)
Target before augmentation: 0.75 hours

Performing stratified sampling...
Sampled 216 files, 0.75 hours
Voice distribution in sampled data:
ameha     91
mekdes    90
gtts      35
Name: voice, dtype: int64

Copying and augmenting audio files...


Processing:   0%|          | 0/216 [00:00<?, ?it/s]

Error processing Dataset/audio/legal_003078_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_002498_mekdes.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_005060_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None


Processing:   5%|▌         | 11/216 [00:00<00:01, 107.58it/s]

Error processing Dataset/audio/legal_001965_mekdes.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_005087_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_004133_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None


...

Error processing Dataset/audio/legal_007042_gtts.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_005668_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_004176_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None


Processing:  99%|█████████▉| 855/863 [00:08<00:00, 93.51it/s]

Error processing Dataset/audio/legal_001317_mekdes.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_004850_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_006912_gtts.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_006091_gtts.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_004186_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_000577_mekdes.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype

Processing: 100%|██████████| 863/863 [00:08<00:00, 105.27it/s]

Error processing Dataset/audio/legal_003176_ameha.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_002804_mekdes.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None
Error processing Dataset/audio/legal_000881_mekdes.mp3: ufunc '_phasor_angles' did not contain a loop with signature matching types <class 'numpy.dtype[float32]'> -> None

Augmented dataset: 1513 files, 5.29 hours
Original: 863 files
Augmented: 650 files

Splitting into train/val/test (80/10/10)...

Dataset created in: Dataset_6.0h
  Train: 1210 samples (4.26h)
  Val: 151 samples (0.51h)
  Test: 152 samples (0.51h)
  Total: 1513 samples (5.29h)

All datasets created successfully!


## 8. Summary Statistics

In [19]:
print("\nDataset Summary:")
print("="*70)

for target_size in TARGET_SIZES_HOURS:
    if target_size in results:
        augmented_df = results[target_size]['augmented_df']
        train_df = results[target_size]['train_df']
        val_df = results[target_size]['val_df']
        test_df = results[target_size]['test_df']
        
        print(f"\nDataset_{target_size}h:")
        print(f"  Total samples: {len(augmented_df)}")
        print(f"  Total duration: {augmented_df['duration'].sum()/3600:.2f} hours")
        print(f"  Train: {len(train_df)} samples ({train_df['duration'].sum()/3600:.2f}h)")
        print(f"  Val: {len(val_df)} samples ({val_df['duration'].sum()/3600:.2f}h)")
        print(f"  Test: {len(test_df)} samples ({test_df['duration'].sum()/3600:.2f}h)")
        print(f"  Voice distribution:")
        for voice, count in augmented_df['voice'].value_counts().items():
            pct = count / len(augmented_df) * 100
            print(f"    {voice}: {count} ({pct:.1f}%)")



Dataset Summary:

Dataset_1.5h:
  Total samples: 378
  Total duration: 1.32 hours
  Train: 302 samples (1.06h)
  Val: 37 samples (0.12h)
  Test: 39 samples (0.14h)
  Voice distribution:
    ameha: 160 (42.3%)
    mekdes: 155 (41.0%)
    gtts: 63 (16.7%)

Dataset_2.0h:
  Total samples: 517
  Total duration: 1.77 hours
  Train: 413 samples (1.41h)
  Val: 51 samples (0.17h)
  Test: 53 samples (0.18h)
  Voice distribution:
    ameha: 238 (46.0%)
    mekdes: 211 (40.8%)
    gtts: 68 (13.2%)

Dataset_4.0h:
  Total samples: 1030
  Total duration: 3.49 hours
  Train: 824 samples (2.74h)
  Val: 103 samples (0.37h)
  Test: 103 samples (0.38h)
  Voice distribution:
    mekdes: 450 (43.7%)
    ameha: 444 (43.1%)
    gtts: 136 (13.2%)

Dataset_6.0h:
  Total samples: 1513
  Total duration: 5.29 hours
  Train: 1210 samples (4.26h)
  Val: 151 samples (0.51h)
  Test: 152 samples (0.51h)
  Voice distribution:
    mekdes: 660 (43.6%)
    ameha: 651 (43.0%)
    gtts: 202 (13.4%)
